In [1]:
import joblib
sclf = joblib.load('Stacking95-80.pkl')

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

from collections import defaultdict

import json

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score

import itertools

import random

from skimage import measure
import dill
import csv


In [3]:
def read_annotation_file(path):
    with open(path) as annotation_file:
        annotation_list = json.load(annotation_file)
    # Transform list of annotations into dictionary
    annotation_dict = {}
    for annotation in annotation_list:
        sequence_id = annotation['sequence_id']
        if sequence_id not in annotation_dict:
            annotation_dict[sequence_id] = {}
        annotation_dict[sequence_id][annotation['frame']] = annotation['object_coords']
    return annotation_dict

train_annotation=read_annotation_file('geoSatellites/train_anno.json')

In [4]:
random.seed(0)

#function to choose a negative sample randomly from an image, with restriction of not being the positive ones or near to them
def random_different_coordinates(coords, size_x, size_y, pad,cond):
    """ Returns a random set of coordinates that is different from the provided coordinates, 
    within the specified bounds.
    The pad parameter avoids coordinates near the bounds."""
    good = False
    while not good:
        good = True
        c1 = random.randint(pad + 1, size_x - (pad + 1))
        c2 = random.randint(pad + 1, size_y -( pad + 1))
        
        #We choose negative samples in a manner that neither positive pixels, nor pixels in the around area of them are chosen
        #That's why we feed with the boolean cond=true if we don't want to choose the neighborhood pixels
        if cond:
            for c in coords:
                coordset_0 = range(int(c[0])-1,int(c[0])+2)
                coordset_1 = range(int(c[1])-1,int(c[1])+2)
                if c1 in coordset_0 and c2 in coordset_1:
                    good = False
                    break
        else:
            for c in coords:
                if c1==c[0] and c2==c[1]:
                    good = False
                    break
    return (c1,c2)

#This function returns the training features we will use (a square array of pixels in area of distance<=radius ) 

def extract_neighborhood(x, y, arr, radius):
    """ Returns a 1-d array of the values within a radius of the x,y coordinates given """
    return arr[(x - radius) : (x + radius + 1), (y - radius) : (y + radius + 1)].ravel()

#check if we can include the pixel, or it's in the edges and we can't have its neighborhood

def check_coordinate_validity(x, y, size_x, size_y, pad):
    """ Check if a coordinate is not too close to the image edge """
    return x >= pad and y >= pad and x + pad < size_x and y + pad < size_y

#Append to training set every positive sample in the given images and a given number of negative samples

def generate_labeled_data(image_path, annotation, nb_false, radius,cond):
    """ For one frame and one annotation array, returns a list of labels 
    (1 for true object and 0 for false) and the corresponding features as an array.
    nb_false controls the number of false samples
    radius defines the size of the sliding window (e.g. radius of 1 gives a 3x3 window)"""
    features,labels = [],[]
    im_array = read_image(image_path)
    # True samples
    for obj in annotation:
        obj = [int(x + .5) for x in obj] #Project the floating coordinate values onto integer pixel coordinates.
        # For some reason the order of coordinates is inverted in the annotation files
        if check_coordinate_validity(obj[1],obj[0],im_array.shape[0],im_array.shape[1],radius):
            features.append(extract_neighborhood(obj[1],obj[0],im_array,radius))
            labels.append(1)
            #features.append(extract_neighborhood(obj[1],obj[0],im_array,radius))
            #labels.append(1)
    # False samples
    for i in range(nb_false):
        c = random_different_coordinates(annotation,im_array.shape[1],im_array.shape[0],radius,cond)
        features.append(extract_neighborhood(c[1],c[0],im_array,radius))
        labels.append(0)
    return np.array(labels),np.stack(features,axis=1)

In [5]:
def generate_labeled_set(annotation_array, path, sequence_id_list, radius, nb_false,cond):
    # Generate labeled data for a list of sequences in a given path
    labels,features = [],[]
    for seq_id in sequence_id_list:
        for frame_id in range(1,6):
            d = generate_labeled_data(f"{path}{seq_id}/{frame_id}.png",
                                    annotation_array[seq_id][frame_id],
                                    nb_false,
                                    radius,cond)
            labels.append(d[0])
            features.append(d[1])
    return np.concatenate(labels,axis=0), np.transpose(np.concatenate(features,axis=1))

In [6]:
def read_image(path):
    return plt.imread(path)


In [7]:
def classify_image(im, model, radius):
    n_features=(2*radius+1)**2 #Total number of pixels in the neighborhood
    feat_array=np.zeros((im.shape[0],im.shape[1],n_features))
    for x in range(radius+1,im.shape[0]-(radius+1)):
        for y in range(radius+1,im.shape[1]-(radius+1)):
            feat_array[x,y,:]=extract_neighborhood(x,y,im,radius)
    all_pixels=feat_array.reshape(im.shape[0]*im.shape[1],n_features)
    pred_pixels=model.predict(all_pixels).astype(np.bool_)
    pred_image=pred_pixels.reshape(im.shape[0],im.shape[1])
    return pred_image
def extract_centroids(pred, bg):
    conn_comp=measure.label(pred, background=bg)
    object_dict=defaultdict(list) #Keys are the indices of the connected components and values are arrrays of their pixel coordinates 
    for (x,y),label in np.ndenumerate(conn_comp):
            if label != bg:
                object_dict[label].append([x,y])
    # Mean coordinate vector for each object, except the "0" label which is the background
    centroids={label: np.mean(np.stack(coords),axis=0) for label,coords in object_dict.items()}
    object_sizes={label: len(coords) for label,coords in object_dict.items()}
    return centroids, object_sizes
def filter_large_objects(centroids,object_sizes, min_size,max_size):
    small_centroids={}
    for label,coords in centroids.items():
            if object_sizes[label] <= max_size and object_sizes[label]>min_size:
                small_centroids[label]=coords
    return small_centroids

def predict_objects(sequence_id, frame_id, model, radius, min_size,max_size):
    print(sequence_id)
    test_image = plt.imread(f"geoSatellites/train/{sequence_id}/{frame_id}.png")
    test_pred=classify_image(test_image, model, radius)
    test_centroids, test_sizes = extract_centroids(test_pred, 0)
    test_centroids = filter_large_objects(test_centroids, test_sizes,min_size, max_size)
    # Switch x and y coordinates for submission
    if len(test_centroids.values()) > 0:
        sub=np.concatenate([c[np.array([1,0])].reshape((1,2)) for c in test_centroids.values()])
        #np array converted to list for json seralization, truncated to the first 30 elements
        return sub.tolist()[0:30]
    else:
        return []

In [8]:
import time
start_time = time.time()


sequencerange = range(1202,1211)
framerange = range(1,6)
sub_list = []
for myseq in sequencerange:
    seq_time = time.time()
    for myframe in framerange:
        fr_time = time.time()
        sub_list.append(predict_objects(myseq,myframe,sclf,3,0,5))
        print('Frame: '+str(time.time()-fr_time))
        #print(len(sub_list))
        #print(sub_list[0:5])
    print('Sequence: '+str(time.time()-seq_time))
submission=[]
for s in range(1202,1211):
    #print(s)
    for fr in range(1,6):
        if s in sequencerange:
            submission.append({"sequence_id" : s, 
                                    "frame" : fr, 
                                    "num_objects" : len(sub_list[(s-1202)*5 + fr-1]), 
                                    "object_coords" : sub_list[(s-1202)*5 + fr-1]})
        else:
            submission.append({"sequence_id" : s,
                                    "frame" : fr,
                                    "num_objects" : 0,
                                    "object_coords" : []})
with open('my_submissionSmall_Stack95-80_train1202-1210flt1-5.json', 'w') as outfile:
    json.dump(submission, outfile)
print(time.time()-start_time)

1202
Frame: 198.48847436904907
1202
Frame: 193.1818130016327
1202
Frame: 193.1761350631714
1202
Frame: 191.89705991744995
1202
Frame: 194.81970715522766
Sequence: 971.5651886463165
1203
Frame: 196.01037168502808
1203
Frame: 197.9352102279663
1203
Frame: 195.36734533309937
1203
Frame: 195.12195992469788
1203
Frame: 196.48712968826294
Sequence: 980.925012588501
1204
Frame: 192.64844489097595
1204
Frame: 192.84836292266846
1204
Frame: 192.7104651927948
1204
Frame: 192.54259514808655
1204
Frame: 192.66643142700195
Sequence: 963.4202969074249
1205
Frame: 193.00304675102234
1205
Frame: 193.66933917999268
1205
Frame: 193.48781085014343
1205
Frame: 194.27647376060486
1205
Frame: 193.2038927078247
Sequence: 967.6445379257202
1206
Frame: 192.31293654441833
1206
Frame: 190.24693036079407
1206
Frame: 192.24490904808044
1206
Frame: 194.6582555770874
1206
Frame: 191.6450653076172
Sequence: 961.1110942363739
1207
Frame: 195.14955234527588
1207
Frame: 195.33100366592407
1207
Frame: 194.1865336894989
1